In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST", one_hot=True)
tf.reset_default_graph()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
n_inputs = 28 # line by line
max_time = 28 # 28 line (output)
lstm_size = 100 # block
n_classes = 10 
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

saver = tf.train.Saver()

def RNN(X, weights, biases):
#     with tf.variable_scope("RNN"):
        # [batch_size, max_time, n_inputs]
    inputs = tf.reshape(X, [-1, max_time, n_inputs])
        
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
#     initial_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
        # final_state[0] cell_state
        # final_state[1] hidden_state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
        
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
        
    return results
        
prediction = RNN(x, weights, biases)

correct_prediction = tf.equal(tf.argmax(prediction,1),tf.arg_max(y,1))

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

update = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("logs", graph=sess.graph)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(update, feed_dict={x:batch_xs, y:batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter "+str(epoch)+" , Testing accuracy= "+str(acc))
        
    saver.save(sess, 'model/lstm1.ckpt')

Instructions for updating:
Use `argmax` instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter 0 , Testing accuracy= 0.7778
Iter 1 , Testing accuracy= 0.8675
Iter 2 , Testing accuracy= 0.9032
Iter 3 , Testing accuracy= 0.9189
Iter 4 , Testing accuracy= 0.9281
Iter 5 , Testing accuracy= 0.9356
